In [ ]:
## Upload files to OpenAI

from openai import OpenAI
client = OpenAI()
import time
import os
import glob

stop = True
for filename in [os.path.abspath(f) for f in glob.iglob('./docx_docs/**/*', recursive=True)]:
    if ".docx" in filename:
        print(filename)
        time.sleep(1)
        file = client.files.create(
            file=open(filename, "rb"),
            purpose='assistants'
        )


In [ ]:
# Fetch file names

from openai import OpenAI
import json
client = OpenAI()

!rm files.json
!curl https://api.openai.com/v1/files -H "Authorization: Bearer $OPENAI_API_KEY" >> files.json
json_file = json.load(open('files.json'))

print(json_file['data'])

In [99]:
# Assistant fetching helper function

from openai import OpenAI
client = OpenAI()

def fetch_all_assistants():
    all_assistants = []
    first_assistants = client.beta.assistants.list(
            order="desc",
            limit=100
        )
    all_assistants.extend(first_assistants.data)
    after_assistant = first_assistants.data[0].id
    while True:
        assistants = client.beta.assistants.list(
            order="desc",
            limit=100,
            after=after_assistant
        )
        all_assistants.extend(assistants.data)
        if len(assistants.data) < 100:
            break
        after_assistant = assistants.data[-1].id
        time.sleep(1)
    return all_assistants

my_assistants = fetch_all_assistants()

In [ ]:
# Remove all assistants from OpenAI

for asst in my_assistants:
    response = client.beta.assistants.delete(asst.id)
    print(response)
    time.sleep(0.1)

In [45]:
# Assistant creation

asst_file = []
for file in json_file['data']:
    if file['filename'].split('.')[-1] == 'docx':
        id = file['id'].split('-')[-1]
        print(file['id'])
        assistant = client.beta.assistants.create(
            name=f"assistant-{id}",
            instructions="""You are a chatbot that answers questions about a document written by Nikola Tesla. You can summarize the document or answer questions about it, if you do not know the answer reply "I'm sorry, I don't know the answer.", don't try to make up something.""",
            model="gpt-3.5-turbo-1106",
            tools=[{"type": "retrieval"}],
            file_ids=[file['id']]
        )
        asst_file.append({'assistant_id': assistant.id, 'file_id': file['id']})
        # Save asst id and file id in dict
        time.sleep(0.1)

file-jcSq4jOaMo1YmKFBvFlYs27T
file-g6BUdzkzDmWtjVwK8fpVeCBp
file-XVWHdXm6JKS45GTGXhea65iM
file-RQyWW6i406BHN484yw0aKuXO
file-DmsyOMmxjd2tj2DPn3Q8bEB5
file-5ny5Pai1i8zNT80jIOow9DQQ
file-3Wq5ItkDO3MEL2vDbJsiaE9t
file-exs1UxzZ6fP0zQCGCD73Tet4
file-VFgFPQJ4ASEtSNZ6Wm5q2Z5w
file-6J8lGw0udn93mVCm8Q3Ehlfe
file-VAEM4xmpbw9aPo8O7XEEqWxh
file-parxM5Lj7NW1uajPMikatkcL
file-QwsZXefqMy7wSMezMsct6jNV
file-M1WeN49qCRVoP3TUi9W3tWRB
file-EYjGl0Jnn0yn2S5aSgMDNIcQ
file-xiKepZxXedVcHjAVtttQPRgp
file-0FuwUxhsR8uXh72OgBOdjloh
file-WDq3V8dDJpI86uLn5IhZTbPI
file-jVHDDR33fbExCAsLEa3ceLot
file-fsL1WqdarpwJ1W7ibKUWVGXh
file-xk4T0CBsT4IwMEZ62El90aMC
file-NtN2BoGhX3pSWN3znQF7aKvw
file-gYSvpNXcqRZmLdaODWN5BdvK
file-2wWwDBOmdmlP0p36PGSSMcih
file-gumJUudxVLxF8N6SXcny6Veo
file-ieYzeZzx8kd0uOZ5fX2i35wl
file-IvZAhObCH5KVabY3u6ZNpet4
file-xbJgvwC96R6ZNiCjW5rBRwpW
file-lGgdTb6urKEKWSmdd9Um1hFc
file-7s0z3F10B25i2mhwSL2lGwjj
file-E4reFqLIPQhGdZzQvUKoXJko
file-uzlHNNmdT7yyaS39KLCjP27S
file-fOaBelEnXRNMo0lcjaaqlhbh
file-V9CKi

In [78]:
asst_file_json = [{'assistant_id': asst.id, 'file_id': asst.file_ids[0] if asst.file_ids else ''} for asst in my_assistants]

In [79]:
len(asst_file_json)

458

In [80]:
import json
with open('asst_file.json', 'w') as f:
    json.dump(asst_file_json, f)

In [54]:
from openai import OpenAI
client = OpenAI()

a = client.files.list()

In [58]:
docx_files = [file for file in a.data if file.filename.split(".")[-1] == "docx"]

In [60]:
len(docx_files)

359